# Hourly dataset for the project

In [1]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


qh_nord.tail()

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,MACROZONA,SBIL_MWH
ORAINI,,
2024-11-27 08:00:00,NORD,164.490
2024-11-27 08:15:00,NORD,105.224
2024-11-27 08:30:00,NORD,9.574
2024-11-27 08:45:00,NORD,-23.560
2024-11-27 09:00:00,NORD,57.032


In [3]:
# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_6412\4018051853.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


,SBIL_MWH
ORAINI,
2024-11-27 05:00:00,433.838
2024-11-27 06:00:00,386.907
2024-11-27 07:00:00,511.406
2024-11-27 08:00:00,255.728
2024-11-27 09:00:00,57.032


In [4]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_6412\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-11-27 05:00:00,369.053,317.147,252.779,66.943
2024-11-27 06:00:00,433.838,369.053,317.147,-217.180
2024-11-27 07:00:00,386.907,433.838,369.053,-347.169


In [5]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,1843.786,0.0,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,1845.050,0.0,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,1845.647,0.0,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,983.111,0.0,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,978.775,0.0,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 19:00:00,41948.262,42783.762,3454.943,7369.242,23603.056,1046.867,2283.798,2506.305,0.0,None,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,39390.746,42283.762,2024-11-26 13:10:41.150
2024-11-27 20:00:00,39561.819,40847.819,3246.019,7029.584,22076.427,1031.956,2177.890,2449.015,0.0,None,...,NaN,0.0,3877.000,0.0,0.0,412.0,2981.0,37454.819,40347.819,2024-11-26 13:10:41.150
2024-11-27 21:00:00,36556.981,37070.981,2967.479,6519.564,20541.635,963.880,2022.610,2258.728,0.0,None,...,NaN,0.0,3384.773,203.6,0.0,363.0,2632.0,34062.006,36556.981,2024-11-26 13:10:41.150


In [6]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


#mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
#mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
#mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
#mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
#mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [7]:
#mi1_volumes_nord.tail(40)

In [8]:
# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16_
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * 
FROM POWER_CURVE_QH_HISTORY 
WHERE CALENDAR_QH_END_DATE > '2019-01-01'
  AND ID_CURVE_SOURCE = 1
"""
power_curve = fetch_db_table_sqlserver16_(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='CALENDAR_QH_END_DATE')

# Group by CALENDAR_QH_END_DATE and sum relevant columns
power_curve = power_curve.groupby('CALENDAR_QH_END_DATE')[['CURVE_VALUE_kWh', 'POWER_VALUE_kW']].sum().reset_index()

# Convert 'CALENDAR_QH_END_DATE' to datetime
power_curve['CALENDAR_QH_END_DATE'] = pd.to_datetime(power_curve['CALENDAR_QH_END_DATE'])

# Change the column name directly using the 'columns' attribute
power_curve.columns = power_curve.columns.str.replace('CALENDAR_QH_END_DATE', 'ORAINI')

# Convert 'ORAINI' to datetime
power_curve['ORAINI'] = pd.to_datetime(power_curve['ORAINI'])

# Set 'ORAINI' as the index for resampling
power_curve.set_index('ORAINI', inplace=True)

# Resample to hourly and sum CURVE_VALUE_kWh and POWER_VALUE_kW
hourly_power_curve = power_curve[['CURVE_VALUE_kWh', 'POWER_VALUE_kW']].resample('H').sum()

# Display the result
hourly_power_curve

c:\imbalance_forecast\src\open_data.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_6412\4073409192.py:35: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_power_curve = power_curve[['CURVE_VALUE_kWh', 'POWER_VALUE_kW']].resample('H').sum()


,CURVE_VALUE_kWh,POWER_VALUE_kW
ORAINI,,
2024-08-26 11:00:00,109670.887,438683.14
2024-08-26 12:00:00,223705.302,894820.32
2024-08-26 13:00:00,224063.234,896252.04
2024-08-26 14:00:00,218228.830,872914.40
2024-08-26 15:00:00,205899.205,823595.87
...,...,...
2024-11-27 05:00:00,103873.991,415495.66
2024-11-27 06:00:00,117144.109,468576.12
2024-11-27 07:00:00,134643.562,538573.83


In [9]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM ENTSOE_DATA" 
entsoe_data = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
entsoe_data = entsoe_data.sort_values(by='ORAINI')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [10]:
entsoe_data = entsoe_data[entsoe_data['DOMAIN'] == 'LOAD']
entsoe_data = entsoe_data.drop(columns=['DOMAIN'])
# Convert ORAINI to datetime using the correct method
entsoe_data['ORAINI'] = pd.to_datetime(entsoe_data['ORAINI'], format='%Y%m%d%H%M')
entsoe_data.set_index('ORAINI', inplace=True)

day_ahead_load = entsoe_data[entsoe_data['SCOPE'] == 'DAY AHEAD']
day_ahead_load = day_ahead_load[['CODZONA', 'VALUE']]
day_ahead_load["FORECAST_TOTAL_LOAD_MW"] = day_ahead_load["VALUE"]
day_ahead_load = day_ahead_load.drop(columns="VALUE")

actual_load = entsoe_data[entsoe_data['SCOPE'] == 'ACTUAL']
actual_load = actual_load[['CODZONA', 'VALUE']]
actual_load["TOTAL_LOAD_MW"] = actual_load["VALUE"]
actual_load = actual_load.drop(columns="VALUE")

entsoe_load = pd.merge(day_ahead_load, actual_load, on=['ORAINI', 'CODZONA'])

subset_zonas = ['NORD']
mnord_load = entsoe_load[entsoe_load['CODZONA'].isin(subset_zonas)]
mnord_load = mnord_load.drop(columns=['CODZONA'])
mnord_load


,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-11-27 04:00:00,14510.0,14268.0
2024-11-27 05:00:00,15826.0,15747.0
2024-11-27 06:00:00,19528.0,19540.0


In [11]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, mgp_volumes_nord, mnord_load, hourly_power_curve] #mi1_volumes_nord
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_h_project = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h_project = df_nord_h_project[df_nord_h_project.index >= '2024-08-27']
df_nord_h_project

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,CURVE_VALUE_kWh,POWER_VALUE_kW
ORAINI,,,,,,,,,,,
2024-08-27 00:00:00,-6.856,-97.279,-51.994,-409.274,-270.368,16126.496,12677.375,17358.0,16029.0,100673.354,402693.14
2024-08-27 01:00:00,35.347,-6.856,-97.279,-51.994,-147.570,15370.564,11892.387,16570.0,15261.0,99141.272,396564.84
2024-08-27 02:00:00,-32.585,35.347,-6.856,-97.279,-214.758,14943.335,11202.301,16123.0,14842.0,97896.639,391586.29
2024-08-27 03:00:00,-117.998,-32.585,35.347,-6.856,21.760,14713.801,10895.785,15720.0,14628.0,97242.570,388970.06
2024-08-27 04:00:00,-297.574,-117.998,-32.585,35.347,-157.722,14760.303,11026.509,15615.0,14826.0,97295.841,389183.08
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 19:00:00,NaN,NaN,NaN,NaN,NaN,23603.056,20197.108,NaN,NaN,NaN,NaN
2024-11-27 20:00:00,NaN,NaN,NaN,NaN,NaN,22076.427,18824.868,NaN,NaN,NaN,NaN
2024-11-27 21:00:00,NaN,NaN,NaN,NaN,NaN,20541.635,17539.446,NaN,NaN,NaN,NaN


In [12]:
df_nord_h_project.tail(48)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,MGP_NORD_PURCHASES,MGP_NORD_SALES,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW,CURVE_VALUE_kWh,POWER_VALUE_kW
ORAINI,,,,,,,,,,,
2024-11-26 00:00:00,-90.602,181.765,242.194,65.250,-60.763,15661.763,11480.177,16027.0,15073.0,140273.218,561092.61
2024-11-26 01:00:00,-63.182,-90.602,181.765,242.194,-419.813,15129.357,10940.358,15275.0,14370.0,124616.631,498466.23
2024-11-26 02:00:00,-30.238,-63.182,-90.602,181.765,-340.447,14841.770,10310.622,14729.0,13885.0,112308.021,449231.82
2024-11-26 03:00:00,-68.431,-30.238,-63.182,-90.602,-210.022,14880.944,10233.124,14609.0,13809.0,106336.876,425347.26
2024-11-26 04:00:00,23.814,-68.431,-30.238,-63.182,-415.579,15105.447,10329.264,14741.0,14062.0,102065.042,408259.90
2024-11-26 05:00:00,66.943,23.814,-68.431,-30.238,-300.661,16057.116,11372.418,16025.0,15462.0,103842.953,415371.53
2024-11-26 06:00:00,-217.180,66.943,23.814,-68.431,-546.326,19360.231,16019.909,19811.0,19358.0,117880.763,471522.77
2024-11-26 07:00:00,-347.169,-217.180,66.943,23.814,-301.155,22879.539,18217.094,23634.0,23335.0,126110.836,504442.86
2024-11-26 08:00:00,-766.554,-347.169,-217.180,66.943,124.695,25000.388,19798.275,25930.0,26246.0,137410.325,549640.47


In [13]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h_project.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_6412\2037195458.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
